In [1]:
%matplotlib notebook

import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import *
import pyvista as pv
from matplotlib.colors import ListedColormap
import time
import vtk

In [2]:
# Load in data
earthquake_data = pv.read('cleaned_earthquake_data.vtk')
earthquake_data

PolyData (0x20a065f8940)
  N Cells:    1
  N Points:   31500
  N Strips:   0
  X Bounds:   -1.272e+07, -1.211e+07
  Y Bounds:   4.404e+06, 5.236e+06
  Z Bounds:   0.000e+00, 6.628e+00
  N Arrays:   31

In [3]:
# Get years without duplicates
active_years = []
for year in earthquake_data.point_data['Year']:
    if year not in active_years:
        active_years.append(year)
active_years.sort()

# Define Utah boundaries by corner points
utah_corners = np_points = np.array([[-114.04172399259161, 41.99372594774304, 0.0], 
                      [-111.0466887774167, 42.0015686697003, 0.0],
                      [-111.04673083487624, 40.997968805895084, 0.0],
                      [-109.05007687780909, 41.000660502664616, 0.0],
                      [-109.04522189716712, 36.999084296202696, 0.0],
                      [-114.05060037628137, 37.00039575681357, 0.0]])
40.76022686400715, -111.89988568426213

utah_cities_coords = np_points = np.array([[-111.89988568426213, 40.76022686400715, 0.0], # Salt Lake City
                      [-111.95532425640559, 41.2074788346799, 0.0], # Ogden
                      [-111.66721654444761, 40.21173939627368, 0.0], # Provo
                      [-109.5578414928903, 38.59039599089006, 0.0], # Moab
                      [-113.0514939220321, 37.683093578904014, 0.0], # Cedar City
                      [-113.57883768492142, 37.0982705156393, 0.0]]) # St. George
utah_cities_names = ['Salt Lake City', 'Ogden', 'Provo', 'Moab', 'Cedar City', 'St. George']

In [4]:
# Get info to build meshes by year
def get_coords_by_year(year):
    lat_lng = []
    indices = np.where(earthquake_data.point_data['Year'] == year)[0]
    for i in indices:
        x, y, z = float(earthquake_data.point_data['Long'][i]), earthquake_data.point_data['Lat'][i], 0
        lat_lng.append((x, y, z))
    lat_lng_np = np.array(lat_lng)        
    return lat_lng_np

def get_all_coords():
    lat_lng = []
    for i in range(len(earthquake_data.point_data['Lat'])):
        x, y, z = float(earthquake_data.point_data['Long'][i]), earthquake_data.point_data['Lat'][i], 0
        lat_lng.append((x, y, z))
    lat_lng_np = np.array(lat_lng)
    return lat_lng_np

def get_mags_by_year(year):
    mags = []
    indices = np.where(earthquake_data.point_data['Year'] == year)[0]
    for i in indices:
        mags.append(earthquake_data.point_data['Mag'][i])
    mags_np = np.array(mags)
    return mags_np.flatten()

def get_all_mags():
    mags = []
    for mag in earthquake_data.point_data['Mag']:
        mags.append(mag)
    mags_np = np.array(mags)
    return mags_np.flatten()

utah_corners = np_points = np.array([[-114.04172399259161, 41.99372594774304, 0.0], 
                      [-111.0466887774167, 42.0015686697003, 0.0],
                      [-111.04673083487624, 40.997968805895084, 0.0],
                      [-109.05007687780909, 41.000660502664616, 0.0],
                      [-109.04522189716712, 36.999084296202696, 0.0],
                      [-114.05060037628137, 37.00039575681357, 0.0]])

In [5]:
# Set up custom cmap, note that we don't have mags that hit the red/green regions, but still display them in our classifications
c1 = np.array([0.99, 0.0, 0.0]) # red
c2 = np.array([0.0, 0.99, 0.0]) # green
c3 = np.array([0.0, 0.0, 0.99]) # blue
c4 = np.array([0.99, 0.99, 0.0]) # yellow
c5 = np.array([0.99, 0.0, 0.99]) # pink
c6 = np.array([0.0, 0.99, 0.99]) # cyan

def get_cmap():
    mapping = np.linspace(earthquake_data.point_data['Mag'].min(), earthquake_data.point_data['Mag'].max(), 256)
    new_colors = np.empty((256, 3))
    new_colors[mapping >= 8.0] = c1
    new_colors[mapping < 7.9] = c2
    new_colors[mapping < 6.9] = c3
    new_colors[mapping < 6.0] = c4
    new_colors[mapping < 5.4] = c5
    new_colors[mapping < 2.5] = c6
    
    mag_classification_cmap = ListedColormap(new_colors)
    lut = pv.LookupTable(cmap=mag_classification_cmap)
    return lut

In [6]:
# Set up legend based on classifications
legend_entries = []
legend_entries.append(['Usually not felt', c6])
legend_entries.append(['Minor damage', c5])
legend_entries.append(['Slight damage', c4])
legend_entries.append(['Lots of damage', c3])
legend_entries.append(['Serious damage', c2])
legend_entries.append(['Destroys entire communities', c1])

In [7]:
# Creates mesh by year and stores coords and mag glyphs in global variables
def create_mag_mesh(year, mode):
    # Create mesh with coords and mag scalars
    mag_mesh = pv.UnstructuredGrid()
    mag_mesh.points = get_coords_by_year(year)
    mag_mesh.point_data['Mag'] = get_mags_by_year(year)
    
    # Select color map
    mode_cmap = ''
    if mode == 'classes':
        mode_cmap = get_cmap()
    else:
        mode_cmap = 'seismic'
    
    # Build coord mesh and mag glyphs
    coords = pl.add_mesh(mag_mesh, point_size=1)
    ball = pv.Sphere(radius=0.05, theta_resolution=35, phi_resolution=35)
    mag_glyphs = mag_mesh.glyph(geom=ball, orient=False, scale="Mag")
    mags = pl.add_mesh(mag_glyphs, cmap=mode_cmap)
    
    # Set visibility to false, will be toggled interactively later
    coords.visibility = False
    mags.visibility = False
    
    # Store meshes by year
    coord_meshes[year] = coords
    mag_meshes[year] = mags
    return

# Only the selected year has visible coords/mags
def move_year_forward():
    # Must be global, won't render otherwise
    global year_index
    global year_text
    year_index += 1
    if year_index > len(active_years) - 1:
        year_index -= 1
        return
    else:
        year = active_years[year_index]
        coord_meshes[year].visibility = True
        mag_meshes[year].visibility = True
        if not year_index == 0:
            prior_year = active_years[year_index - 1]
            coord_meshes[prior_year].visibility = False
            mag_meshes[prior_year].visibility = False
        pl.remove_actor(year_text)
        year_text = pl.add_text('Year: ' + str(year))
        pl.update()

def move_year_backward():
    global year_index
    global year_text
    
    year_index -= 1
    if year_index < 0:
        year_index += 1
        return
    else:
        year = active_years[year_index]
        coord_meshes[year].visibility = True
        mag_meshes[year].visibility = True
        if not year_index == len(active_years) - 1:
            prior_year = active_years[year_index + 1]
            coord_meshes[prior_year].visibility = False
            mag_meshes[prior_year].visibility = False
        pl.remove_actor(year_text)
        year_text = pl.add_text('Year: ' + str(year))
        pl.update()
        
def toggle_cities():
    global cities_actor
    if cities_actor.GetVisibility() == 1:
        cities_actor.VisibilityOff()
    else:
        cities_actor.VisibilityOn()
    pl.update()

In [13]:
# Mode refers to color map, either use 'classes' for custom classification, otherwise it will use 'seismic' (blue to red)
def run_vis(mode='classes'):
    global year_text
    global cities_actor
    # Display approximate boundaries for Utah
    poly_line = pv.MultipleLines(points=utah_corners)
    pl.add_mesh(poly_line, line_width=2)
    line = pv.Line(pointa=utah_corners[0], pointb=utah_corners[5])
    pl.add_mesh(line, line_width=2, color='white')
    
    # Display Utah cities
    cities = pv.PolyData(utah_cities_coords)
    cities['City Names'] = utah_cities_names
    cities_actor = pl.add_point_labels(cities, 'City Names', point_size = 15, font_size=18, 
                                       shape_color='red', always_visible=True)
    
    # Only add legend if we're using the classification mode
    if mode == 'classes':
        pl.add_legend(legend_entries)

    # Load points for all years, but we only display one at a time
    for year in active_years:
        create_mag_mesh(year, mode)
        
    year_text = pl.add_text("")
    
    pl.add_title("Utah Earthquakes 1850-2016", font_size=14)
    pl.add_text("f: next year\nb: prior year\nc: toggle cities", "lower_left", font_size=12)
        
    # Interactive events to move through years
    pl.add_key_event("f", move_year_forward)
    pl.add_key_event("b", move_year_backward)
    pl.add_key_event("c", toggle_cities)
    
    pl.show(cpos='xy')

In [18]:
# Visualization showing all earthquakes, mapped by class, and notable cities
def run_all_year_vis():
    pl = pv.Plotter(notebook=False)
    # Display approximate boundaries for Utah
    poly_line = pv.MultipleLines(points=utah_corners)
    pl.add_mesh(poly_line, line_width=2)
    line = pv.Line(pointa=utah_corners[0], pointb=utah_corners[5])
    pl.add_mesh(line, line_width=2, color='white')
    
    # Display Utah cities
    cities = pv.PolyData(utah_cities_coords)
    cities['City Names'] = utah_cities_names
    cities_actor = pl.add_point_labels(cities, 'City Names', point_size = 15, font_size=18, 
                                       shape_color='red', always_visible=True)

    # Convert data
    mag_mesh = pv.UnstructuredGrid()
    mag_mesh.points = get_all_coords()
    mag_mesh.point_data["Mag"] = get_all_mags()

    # Plot earthquakes
    coords = pl.add_mesh(mag_mesh, point_size=1)
    ball = pv.Sphere(radius=0.05, theta_resolution=35, phi_resolution=35)
    mag_glyphs = mag_mesh.glyph(geom=ball, orient=False, scale="Mag")
    mags = pl.add_mesh(mag_glyphs, cmap=get_cmap())

    pl.add_legend(legend_entries)


    pl.add_title("Utah Earthquakes 1850-2016", font_size=14)

    pl.show(cpos='xy')

In [20]:
# Values used globally, need to reset each run
year_index = -1
coord_meshes = {}
mag_meshes = {}
cities_actor = vtk.vtkActor2D()

pl = pv.Plotter(notebook=False)

run_vis()

In [21]:
# Values used globally, need to reset each run
year_index = -1
coord_meshes = {}
mag_meshes = {}
pl = pv.Plotter(notebook=False)

run_all_year_vis()